In [1]:
import os
import json
import csv
import pandas as pd
import numpy as np
import math

In [2]:
results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_10_22_2023_merged_exp6.csv"

In [3]:
exp_df = pd.read_csv(results_csv_file)  

In [4]:
included_exp_names = exp_df['exp'].tolist()

In [5]:
def get_base_exp_name(exp_name):
    exp_field = "exp"
    exp_field_len_w_index = len(exp_field)+1
    exp_field_index = exp_name.find(exp_field)
    exp_name = exp_name[:exp_field_index]+exp_name[exp_field_index+exp_field_len_w_index:]
    return exp_name

In [6]:
exp_groups = dict()

for exp_name in included_exp_names:
    base_exp_name = get_base_exp_name(exp_name)
    if base_exp_name in exp_groups:
        exp_groups[base_exp_name] += [exp_name]
    else:
        exp_groups[base_exp_name] = [exp_name]

In [7]:
exp_length = 3
num_rounds = 6
round_strings =  ["round" + str(round_num) + "_test" for round_num in range(num_rounds)]
base_exp_names = list(exp_groups.keys())

In [8]:
results_dict = {}

for base_exp_name, exp_group in exp_groups.items():
    exp_dict = {}
    for i, run in enumerate(exp_group):
        sub_df = exp_df.loc[exp_df['exp'] == run]
        for round_string in round_strings:
            val = sub_df[round_string].iloc[0]
            if not math.isnan(val):
                if round_string not in exp_dict:
                    exp_dict[round_string] = [float(val)]
                else:
                    exp_dict[round_string] += [float(val)]
    results_dict[base_exp_name] = exp_dict
    print(base_exp_name)

DMPLS__coreset_cl_bs8_mdl_ft_v6
DMPLS__coreset_mdl_ft_v6
DMPLS__coreset_v6
DMPLS__coreset_cl_bs8_v6
DMPLS__coreset_cl_bs8_fuse10_v6
DMPLS__coreset_preds_v6
DMPLS__coreset_cl_bs8_preds_v6


In [9]:
index = pd.Index(base_exp_names, name="exp_name")
round_counts_strings = [[round_string, f"round{i}_count"] for i, round_string in enumerate(round_strings)]
df = pd.DataFrame(index=index, columns=sum(round_counts_strings, []))

In [10]:
for base_exp_name, results in results_dict.items():
    sub_df = df.loc[base_exp_name]
    results = results_dict[base_exp_name]
    for round_string, count_string in round_counts_strings:
        if round_string in results:
            round_results = results[round_string]
            num_results = len(round_results)
            mean_result = np.mean(round_results)
            std_results = np.std(round_results, ddof=1)
            mean_string = '%.3f' % mean_result
            std_string = '%.3f'% std_results
            sub_df[round_string] = f"{mean_string}+/-{std_string}"
            sub_df[count_string] = num_results

In [11]:
df

,round0_test,round0_count,round1_test,round1_count,round2_test,round2_count,round3_test,round3_count,round4_test,round4_count,round5_test,round5_count
exp_name,,,,,,,,,,,,
DMPLS__coreset_cl_bs8_mdl_ft_v6,0.273+/-0.028,3,0.673+/-0.086,3,0.793+/-0.013,3,0.824+/-0.006,3,0.842+/-0.003,3,0.861+/-0.001,3
DMPLS__coreset_mdl_ft_v6,0.306+/-0.063,3,0.668+/-0.053,3,0.798+/-0.002,3,0.830+/-0.002,3,0.839+/-0.003,3,0.864+/-0.006,3
DMPLS__coreset_v6,0.277+/-0.039,3,0.740+/-0.020,3,0.790+/-0.011,3,0.841+/-0.007,3,0.851+/-0.010,3,0.868+/-0.006,3
DMPLS__coreset_cl_bs8_v6,0.306+/-0.074,3,0.691+/-0.027,3,0.785+/-0.018,3,0.833+/-0.012,3,0.843+/-0.015,3,0.869+/-0.002,3
DMPLS__coreset_cl_bs8_fuse10_v6,0.357+/-0.098,2,0.761+/-0.018,2,0.803+/-0.010,2,0.839+/-0.002,2,0.846+/-0.002,2,0.873+/-0.001,2
DMPLS__coreset_preds_v6,0.299+/-0.065,4,0.704+/-0.052,4,0.741+/-0.034,4,0.808+/-0.021,4,0.842+/-0.007,4,0.857+/-0.002,4
DMPLS__coreset_cl_bs8_preds_v6,0.244+/-0.031,3,0.584+/-0.029,3,0.733+/-0.056,3,0.812+/-0.035,3,0.834+/-0.018,3,0.863+/-0.004,3
